# Read file from db

In [5]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM Store")

    rows = cur.fetchall()
    return rows
        
database = r"/Users/yumengluo/Desktop/claire/Columbia/6998/6998-team/cc6998-grp-prj/data/stores.db"

# create a database connection
conn = create_connection(database)
with conn:

    print("Print all stores")
    rows = select_all_tasks(conn)
    print(len(rows))
    

Print all stores
5280


In [12]:
rows[0:10]

[('JOHN GLENN', 1, 'Wegmans Store', 43.13316, -76.2232),
 ('CHERRY HILL WINE', 2, 'Wine Store', 39.92248, -75.03552),
 ('EASTWAY', 3, 'Wegmans Store', 43.19323, -77.5043),
 ('FAIRPORT', 4, 'Wegmans Store', 43.1019, -77.4825),
 ('WOODBRIDGE WINE', 5, 'Wine Store', 40.5488, -74.2968),
 ('NEWARK', 6, 'Wegmans Store', 43.0452, -77.1139),
 ('DULLES', 7, 'Wegmans Store', 39.00691, -77.44081),
 ('MT. LAUREL', 8, 'Wegmans Store', 39.96771, -74.91507),
 ('OCEAN', 9, 'Wegmans Store', 40.239108, -74.03991),
 ('CHERRY HILL', 10, 'Wegmans Store', 39.92248, -75.03552)]

# Create table

In [9]:
!aws dynamodb create-table --table-name stores --key-schema         AttributeName=id,KeyType=HASH --provisioned-throughput         ReadCapacityUnits=10,WriteCapacityUnits=5    --attribute-definitions         AttributeName=id,AttributeType=S 

{
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "id",
                "AttributeType": "S"
            }
        ],
        "TableName": "stores",
        "KeySchema": [
            {
                "AttributeName": "id",
                "KeyType": "HASH"
            }
        ],
        "TableStatus": "CREATING",
        "CreationDateTime": "2021-12-01T19:42:09.834000-05:00",
        "ProvisionedThroughput": {
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 5
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:us-east-1:859913276865:table/stores",
        "TableId": "7876ec81-3cb9-46a8-ac2e-b03cc73f4f08"
    }
}


# Put item

In [19]:
import datetime 
from pytz import timezone
from decimal import Decimal
import json
def generate_string(store, i):
    now = datetime.datetime.now(timezone('US/Eastern')) 
    
    d = {}
    name,num,store_name,lat,lon = store
    d['name'] ={'S': name}
    d['brand_id'] = {'S':str(num)}
    d['id'] = {'S':store_name+"-"+str(num)+"-"+name}
    d['brand_name'] = {'S':store_name}
    d['lat'] = {'S':str(lat)}
    d['lon'] = {'S':str(lon)}
    d['insertedAtTimestamp']={'S':now.strftime('%Y-%m-%dT%H:%M:%S-%Z')}
    s = json.dumps(d)
    s = '\''+s+'\''
    return s


In [ ]:
count = 0
for store in rows:
    
    attribute_s = generate_string(store, count )
    !aws dynamodb put-item --table-name stores --item $attribute_s
    
    count += 1